### Setup

In [47]:
# general
import numpy as np
import pandas as pd
import csv

# tokenization
import json
import MeCab
import import_ipynb
import thesis_preprocess
from stopwords_ja import stop_words
from stopwords_slothlib import stop_words_2

# lda topic modeling
import gensim, logging
import gensim.corpora as corpora
from gensim.models import CoherenceModel
import pyLDAvis
import pyLDAvis.gensim

importing Jupyter notebook from thesis_preprocess.ipynb


<string>:23: DeprecationWarning: invalid escape sequence '\('
<string>:31: DeprecationWarning: invalid escape sequence '\s'


### Preprocessing and Tokenization

In [41]:
# tokenize cleaned tweets into words
def tokenize(text):
    mt = MeCab.Tagger("-d /usr/local/lib/mecab/dic/mecab-ipadic-neologd")
    parsed = mt.parseToNode(text)
    components = []
    
    while parsed:
        word = parsed.surface
        pos = parsed.feature.split(",")[0]

        # for lda, we only want nouns, verbs, adjectives
        include_pos = ["名詞", "動詞", "形容詞"]
        if pos in include_pos: components.append(word)
        parsed = parsed.next
    
    # remove stopwords
    components = [token for token in components if ((not token in stop_words) and (not token in stop_words_2))]
    
    return components

In [11]:
# run preprocessing and tokenization for tweets from given .txt file
def preprocess_tokenize_all(filename, year):
    # store results and exception tweets
    tokens = []
    retweets = []
    not_parsed = []

    # iterate through tweets, preprocess and tokenize
    with open(filename, 'r') as file:
        for line in file:
            tweet = json.loads(line)
            print(int(year) + 1)
            if line == None or tweet == None:
                not_parsed.append((line, tweet))
                print("Parsing error: ", line, tweet)
            elif tweet['retweetedTweet']:
                retweets.append(tweet)
                print("Retweet: ", tweet['id'])
            # filter out 2024 sponsored(?) tweets
            elif int(tweet['date'].split("-")[0]) > int(year) + 1: 
                tweet_text = tweet['rawContent'] # note: need other prop for over 140 char?
                processed = preprocess(tweet_text)            
                components = tokenize(processed)
                tokens.append(components)

    file.close()
    return tokens, retweets, not_parsed

In [12]:
# # run for 2015
# tokens_2015, retweets_2015, not_parsed_2015 = preprocess_tokenize_all("2015")

# # did we get retweets or errors?
# print(len(retweets_2015))
# print(len(not_parsed_2015))

In [ ]:
# run for 2022
token_tweets_2022, retweets_2022, not_parsed_2022 = preprocess_tokenize_all("2022")

# did we get retweets or errors?
print(len(retweets_2022))
print(len(not_parsed_2022))

In [ ]:
# pass in filename you want to save data as, including '.csv'
def save_to_csv(tweet_tokens, filename):
    f = open(filename, 'w')
    writer = csv.writer(f)
    for tweet in tweet_tokens:
        writer.writerow(tweet)
    f.close()

In [14]:
# pass in filename of csv you want to load, including '.csv'
def load_from_csv(filename):
    with open(filename, newline='') as f:
        reader = csv.reader(f)
        tweet_tokens = list(reader)
    return tweet_tokens

In [55]:
# https://lda.readthedocs.io/en/latest/getting_started.html
# https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/
# https://github.com/deankuo/Japan-Manifesto-Classification/blob/main/topic_modeling.ipynb
# https://github.com/m3yrin/NTM/blob/master/LDA_jp.ipynb
# https://tdual.hatenablog.com/entry/2018/04/09/133000#1LDA%E3%81%AE%E5%89%8D%E3%81%AB%E3%83%88%E3%83%94%E3%83%83%E3%82%AF%E3%83%A2%E3%83%87%E3%83%AB%E3%81%A8%E3%81%AF

In [48]:
# train and save word2vec model for given year
def run_lda(filename, tweet_tokens, no_below=5, no_above=0.2):
    # set up dictionary
    dict = corpora.Dictionary(tweet_tokens)
    dict.filter_extremes(no_below, no_above)
    dict.compactify()

    # set up corpus
    corpus = [dict.doc2bow(w) for w in tweet_tokens]
    test_size = int(len(corpus) * 0.1)
    test_corpus = corpus[:test_size]
    train_corpus = corpus[test_size:]   

    # train and save lda model
    logging.basicConfig(format='%(message)s', level=logging.INFO)
    lda = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=dict, num_topics=20, passes=10, update_every=5)
    lda.save("save_lda_model_" + filename)
    
    return lda, dict, corpus, train_corpus, test_corpus

In [42]:
lda_2022_gen = gensim.models.LdaModel.load("save_lda_model_2022")

loading LdaModel object from thesis_lda_model_2022
loading expElogbeta from thesis_lda_model_2022.expElogbeta.npy with mmap=None
setting ignored attribute id2word to None
setting ignored attribute dispatcher to None
setting ignored attribute state to None
LdaModel lifecycle event {'fname': 'thesis_lda_model_2022', 'datetime': '2024-03-17T10:28:38.384616', 'gensim': '4.3.2', 'python': '3.11.5 (main, Aug 24 2023, 15:18:16) [Clang 14.0.3 (clang-1403.0.22.14.1)]', 'platform': 'macOS-14.3.1-x86_64-i386-64bit', 'event': 'loaded'}
loading LdaState object from thesis_lda_model_2022.state
LdaState lifecycle event {'fname': 'thesis_lda_model_2022.state', 'datetime': '2024-03-17T10:28:38.407748', 'gensim': '4.3.2', 'python': '3.11.5 (main, Aug 24 2023, 15:18:16) [Clang 14.0.3 (clang-1403.0.22.14.1)]', 'platform': 'macOS-14.3.1-x86_64-i386-64bit', 'event': 'loaded'}


In [49]:
# examine topics
def get_topic_words(lda, topic_id):
    for t in lda.get_topic_terms(topic_id):
        print("{}: {}".format(d[t[0]], t[1]))

def examine_topics(lda):
    for t in range(10):
        print("Topic # ",t)
        get_topic_words(lda, t)
        print("\n")

In [50]:
def analyze_train_test_results(lda, train_corpus, test_corpus):
    # look at train set results
    N = sum(count for doc in train_corpus for _, count in doc)
    print("# of words in train corpus: ",N)
    perplexity = np.exp2(-lda.log_perplexity(train_corpus))
    print("perplexity(train):", perplexity,"\n")

    # look at test set results
    N = sum(count for doc in test_corpus for _, count in doc)
    print("# of words in test corpus: ",N)
    perplexity = np.exp2(-lda.log_perplexity(test_corpus))
    print("perplexity(test):", perplexity)

In [51]:
def analyze_overall_results(lda, corpus):
    # look at overall perplexity and coherence score
    print('\nPerplexity: ', lda.log_perplexity(corpus))     # lower is better

    coherence_model_lda = CoherenceModel(model=lda, texts=token_tweets_2022, dictionary=d, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    print('\nCoherence Score: ', coherence_lda)     # higher is better

In [44]:
def visualize_topics(lda, corpus, dict):
    pyLDAvis.enable_notebook()
    vis = pyLDAvis.gensim.prepare(lda, corpus, dict)
    return vis

In [38]:
# https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/
def format_topics_sentences(lda, corpus, tweet_tokens):
    sent_topics_df = pd.DataFrame()

    # get main topic in each document
    for i, row in enumerate(lda[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # for each document, get dominant topic, perc contribution and keywords 
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = lda.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = pd.concat([sent_topics_df,pd.Series([int(topic_num), round(prop_topic,4), topic_keywords])], ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # add original text to the end of the output
    contents = pd.Series(tweet_tokens)
    sent_topics_df = contents

    # TODO: debug this line
    # sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

In [ ]:
# TODO: run this once we collect a keyword dataset

df_topic_sents_keywords = format_topics_sentences(ldamodel=lda, corpus=corpus, texts=token_tweets_2022)

# format and show
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
df_dominant_topic.head(10)

In [ ]:
# group top 5 sentences under each topic
sent_topics_sorteddf = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf = pd.concat([sent_topics_sorteddf, 
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], 
                                            axis=0)

# reset index    
sent_topics_sorteddf.reset_index(drop=True, inplace=True)

# format and show
sent_topics_sorteddf.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"]
sent_topics_sorteddf.head()

In [ ]:
# num documents per topic
topic_counts = df_topic_sents_keywords['Dominant_Topic'].value_counts()

# percentage documents for each topic
topic_contribution = round(topic_counts/topic_counts.sum(), 4)

# topic number, keywords
topic_num_keywords = df_topic_sents_keywords[['Dominant_Topic', 'Topic_Keywords']]

# concatenate column-wise
df_dominant_topics = pd.concat([topic_num_keywords, topic_counts, topic_contribution], axis=1)

# add column names
df_dominant_topics.columns = ['Dominant_Topic', 'Topic_Keywords', 'Num_Documents', 'Perc_Documents']

# show
df_dominant_topics